First load the dataset into program

In [1]:
from csv import reader

file = "abalone_original.csv"
attribute = list()
dataset = list()
data_count = 0
with open(file,'r') as abalone_data:
    data_reader = reader(abalone_data)
    for row in data_reader:
        if not row:
            continue
        if data_count == 0:
            attribute = row
            data_count += 1
        else:
            dataset.append(row)
            data_count += 1
data_count -= 1

print("Total Data : ", data_count)

print("\nAttribute :")
for i in attribute:
    print(i)

print("\nData :")
for i in range(5):
    print(dataset[i])
print("...")

Total Data :  4177

Attribute :
sex
length
diameter
height
whole-weight
shucked-weight
viscera-weight
shell-weight
rings

Data :
['M', '91', '73', '19', '102.8', '44.9', '20.2', '30', '15']
['M', '70', '53', '18', '45.1', '19.9', '9.7', '14', '7']
['F', '106', '84', '27', '135.4', '51.3', '28.3', '42', '9']
['M', '88', '73', '25', '103.2', '43.1', '22.8', '31', '10']
['I', '66', '51', '16', '41', '17.9', '7.9', '11', '7']
...


As the previous results, it's clear that all features have Gaussian Distribution. Next clean the data so that all data become float type.

In [2]:
for row in range(len(dataset)):
    for feature in range(1, len(dataset[row])):
        dataset[row][feature] = float(dataset[row][feature].strip())
        
print("Converted Data :")
for i in range(5):
    print(dataset[i])
print("...")

Converted Data :
['M', 91.0, 73.0, 19.0, 102.8, 44.9, 20.2, 30.0, 15.0]
['M', 70.0, 53.0, 18.0, 45.1, 19.9, 9.7, 14.0, 7.0]
['F', 106.0, 84.0, 27.0, 135.4, 51.3, 28.3, 42.0, 9.0]
['M', 88.0, 73.0, 25.0, 103.2, 43.1, 22.8, 31.0, 10.0]
['I', 66.0, 51.0, 16.0, 41.0, 17.9, 7.9, 11.0, 7.0]
...


We will use 5-fold cross validation to evaluate this model.

In [3]:
from random import randrange

n_fold = 5
fold_size = int(len(dataset) / 5)
data_split = list()
dataset_copy = list(dataset)

for _ in range(n_fold):
    fold = list()
    while len(fold) < fold_size:
        index = randrange(len(dataset_copy))
        fold.append(dataset_copy.pop(index))
    data_split.append(fold)

train_set = list()
for fold in data_split:
    set = list()
    train = list()
    val = list()
    

With the set of data we got from folding, we will make a collection of training set and test set.

In [4]:
train_collection = list()
test_collection = list()
test_class_collection = list()

for i, fold in enumerate(data_split):
    train_set = list(data_split)
    train_set.remove(fold)
    train_set = sum(train_set, [])
    test_set = list()
    test_set_class = list()
    for row in fold:
        row_copy = list(row)
        test_set_class.append(row_copy[0])
        test_set.append(row_copy[1:])
    
    train_collection.append(train_set)
    test_collection.append(test_set)
    test_class_collection.append(test_set_class)

for i in range(n_fold) :
    print(f"Data {i+1}")
    print("Training Set")
    for j in range(5):
        print(train_collection[i][j])
    print("...")

    print("Test Set")
    for j in range(5):
        print(test_collection[i][j])
    print("...\n")
    
    

Data 1
Training Set
['M', 120.0, 93.0, 31.0, 252.4, 124.9, 49.1, 66.0, 10.0]
['F', 112.0, 92.0, 36.0, 194.0, 68.4, 39.2, 71.0, 12.0]
['F', 110.0, 76.0, 33.0, 241.0, 108.6, 58.8, 66.9, 10.0]
['M', 106.0, 83.0, 26.0, 168.5, 55.0, 38.9, 53.0, 20.0]
['F', 124.0, 95.0, 32.0, 264.9, 137.3, 46.6, 65.5, 9.0]
...
Test Set
[100.0, 77.0, 24.0, 103.2, 39.4, 26.1, 33.0, 8.0]
[140.0, 110.0, 39.0, 324.9, 135.0, 69.4, 107.0, 13.0]
[97.0, 79.0, 32.0, 132.0, 49.5, 25.6, 47.0, 14.0]
[105.0, 75.0, 24.0, 126.3, 60.9, 22.8, 38.0, 9.0]
[149.0, 117.0, 38.0, 393.2, 168.7, 87.4, 117.1, 18.0]
...

Data 2
Training Set
['I', 100.0, 77.0, 24.0, 103.2, 39.4, 26.1, 33.0, 8.0]
['M', 140.0, 110.0, 39.0, 324.9, 135.0, 69.4, 107.0, 13.0]
['F', 97.0, 79.0, 32.0, 132.0, 49.5, 25.6, 47.0, 14.0]
['I', 105.0, 75.0, 24.0, 126.3, 60.9, 22.8, 38.0, 9.0]
['F', 149.0, 117.0, 38.0, 393.2, 168.7, 87.4, 117.1, 18.0]
...
Test Set
[120.0, 93.0, 31.0, 252.4, 124.9, 49.1, 66.0, 10.0]
[112.0, 92.0, 36.0, 194.0, 68.4, 39.2, 71.0, 12.0]
[11

Then divide the training set by its class.

In [5]:
for i, training_copy in enumerate(train_collection):
    separated_data = dict()
    for row in training_copy:
        label = row[0]
        value = row[1:]
        if label not in separated_data:
            separated_data[label] = list()
        separated_data[label].append(value)
    train_collection[i] = separated_data

for i, data in enumerate(train_collection) :
    print(f"Data {i+1}")
    print("Training Set")
    for class_, value in data.items():
        print(f"[{class_}]")
        if len(value) >= 5:
            for j in range(5):
                print(value[j])
        else:
            for j in range(len(value)):
                print(value[j])
    print("...")

Data 1
Training Set
[M]
[120.0, 93.0, 31.0, 252.4, 124.9, 49.1, 66.0, 10.0]
[106.0, 83.0, 26.0, 168.5, 55.0, 38.9, 53.0, 20.0]
[122.0, 73.0, 31.0, 215.3, 97.6, 49.8, 54.0, 9.0]
[132.0, 99.0, 39.0, 325.5, 118.8, 71.9, 97.0, 10.0]
[98.0, 77.0, 25.0, 121.8, 61.3, 19.2, 35.5, 8.0]
[F]
[112.0, 92.0, 36.0, 194.0, 68.4, 39.2, 71.0, 12.0]
[110.0, 76.0, 33.0, 241.0, 108.6, 58.8, 66.9, 10.0]
[124.0, 95.0, 32.0, 264.9, 137.3, 46.6, 65.5, 9.0]
[135.0, 106.0, 39.0, 299.7, 124.0, 75.0, 85.0, 9.0]
[126.0, 101.0, 33.0, 213.0, 91.9, 43.2, 63.0, 12.0]
[I]
[57.0, 43.0, 12.0, 18.7, 6.2, 4.6, 6.0, 6.0]
[87.0, 67.0, 22.0, 76.6, 31.1, 13.5, 27.0, 12.0]
[61.0, 43.0, 13.0, 21.5, 8.8, 4.1, 7.6, 5.0]
[54.0, 38.0, 16.0, 16.2, 5.3, 3.9, 6.0, 6.0]
[101.0, 77.0, 25.0, 119.2, 49.0, 19.4, 42.0, 9.0]
...
Data 2
Training Set
[I]
[100.0, 77.0, 24.0, 103.2, 39.4, 26.1, 33.0, 8.0]
[105.0, 75.0, 24.0, 126.3, 60.9, 22.8, 38.0, 9.0]
[85.0, 68.0, 20.0, 74.2, 30.0, 17.3, 23.0, 8.0]
[104.0, 77.0, 23.0, 116.2, 51.1, 31.2, 28.6, 1

Check the disttibution of each feature in each class.

In [6]:
distribution = list()
row_copy = train_collection[0]['M'][0]

for feature in row_copy:
    if type(feature) == int or float:
        distribution.append("continuous")
    else:
        distribution.append("discrete")

print(distribution)

['continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous']


As we can see all features have continuous distribution. So we will use Gaussian Probability Density Function to calculate the feature likelihood.

First step is to measure the statistics (mean and standard deviation) for each feature.

In [7]:
from math import sqrt
from math import exp
from math import pi

statistics = list()

def continuous_convert(dataset):
    convert_data = list()
    for data in dataset:
        convert_data.append(float(data))
    return convert_data

def mean(feature):
    return (sum(feature) / float(len(feature)))

def stdv(feature, mean):
    variance = sum([(x - mean)**2 for x in feature]) / float(len(feature)-1)
    return (sqrt(variance))

for dataset in train_collection:
    data_statistics = dict()
    for class_, data in dataset.items():
        data_statistics[class_] = list()
        
        for feature in zip(*data):
            measurement = list()
            convert_feature = continuous_convert(feature)
            avg = mean(convert_feature)
            dev = stdv(convert_feature, avg)
            
            measurement.append(avg)
            measurement.append(dev)
            measurement.append(len(feature))
            
            data_statistics[class_].append(measurement)
            
    statistics.append(data_statistics)
    
for i, stat in enumerate(statistics):
    print(f"Data {i + 1}")
    for class_, data in stat.items():
        print(f"[{class_}]")
        print(data)
    print("\n")

Data 1
[M]
[[112.22129086336966, 20.542956695879596, 1193], [87.77032690695725, 16.905507957638935, 1193], [30.354568315171836, 7.047479058221862, 1193], [197.22414082145823, 93.00156699600122, 1193], [85.83880972338643, 43.789636437407296, 1193], [42.923973176865, 20.860847579075013, 1193], [56.377367979882656, 26.016168645404356, 1193], [10.720033528918693, 2.9854922135661512, 1193]]
[F]
[[115.65352112676057, 17.335509140422836, 1065], [90.84507042253522, 14.221906637367267, 1065], [31.55211267605634, 8.394040903763758, 1065], [208.2143661971835, 86.32932741972284, 1065], [88.80882629107975, 40.15501491439453, 1065], [45.91633802816907, 19.57053648297601, 1065], [60.1556807511737, 25.180481762223767, 1065], [11.145539906103286, 3.151029387024047, 1065]]
[I]
[[85.1561922365989, 21.794401412830403, 1082], [64.92698706099816, 17.609738706090425, 1082], [21.475046210720887, 6.362603671730947, 1082], [84.84103512014782, 56.312563341113055, 1082], [37.56571164510161, 25.303759376370817, 10

Then create Gaussian PDF.

In [8]:
def continuous_probability(value, mean, stdv):
    exponent = exp(-((value - mean)**2 / (2 * stdv**2)))
    return (1 / (sqrt((2 * pi) * (stdv**2)))) * exponent

Next step is testing our model with test data and get a set of probabilities for each class.

In [9]:
for collecPtr, test_data in enumerate(test_collection):
    total = len(dataset)
    
    for dataPtr, data in enumerate(test_data):
        probabilities = dict()
        
        for class_, stat in statistics[collecPtr].items():
            probabilities[class_] = stat[0][2] / float(total)
            
            for feature_ptr, feature in enumerate(data):
                probabilities[class_] *= continuous_probability(feature
                                                               ,stat[feature_ptr][0]
                                                               ,stat[feature_ptr][1])
        
        test_collection[collecPtr][dataPtr] = probabilities

for i, data in enumerate(test_collection):
    print(f"Data {i+1}")
    print("Test Set")
    for j in range(5):
        print(data[j])
    print("...\n")

Data 1
Test Set
{'M': 7.836196187602342e-13, 'F': 2.5724621487099925e-13, 'I': 5.206702167991545e-11}
{'M': 1.3115829796488392e-14, 'F': 3.1189673252913343e-14, 'I': 4.537441320149523e-32}
{'M': 2.0545616518130565e-12, 'F': 1.1458800671134238e-12, 'I': 2.1498245864904232e-13}
{'M': 1.9593405571423156e-12, 'F': 8.149924827791228e-13, 'I': 2.1233510447419965e-11}
{'M': 2.4829319443479858e-18, 'F': 4.723500073393345e-18, 'I': 3.191727910546518e-45}
...

Data 2
Test Set
{'I': 2.907913338516891e-19, 'M': 7.0812341464234565e-12, 'F': 1.1458721293152168e-11}
{'I': 1.9940599381499953e-16, 'M': 7.575284120410799e-12, 'F': 1.3145655151414366e-11}
{'I': 1.0346943836048116e-18, 'M': 6.169312697648551e-12, 'F': 7.093115193625084e-12}
{'I': 3.94063843773173e-18, 'M': 8.008372201729765e-14, 'F': 1.1078721863039235e-13}
{'I': 2.857524635682692e-20, 'M': 3.747515239410549e-12, 'F': 5.8554468495311435e-12}
...

Data 3
Test Set
{'I': 8.509567710692997e-11, 'M': 1.2824462140254888e-14, 'F': 8.018462007251

After calculating probabilities for all classes, we need to determine which class has teh largest value among others and set that class to be the predicted class for each row.

In [10]:
for collecPtr, test_data in enumerate(test_collection):
    for dataPtr, data in enumerate(test_data):
        best_class, best_prob = None, -1
        for class_, prob in data.items():
            if prob > best_prob:
                best_prob = prob
                best_class = class_
        
        test_collection[collecPtr][dataPtr] = best_class

for i, data in enumerate(test_collection):
    print(f"Data {i+1}")
    print("Test Set")
    for j in range(5):
        print(data[j])
    print("...\n")

Data 1
Test Set
I
F
M
I
F
...

Data 2
Test Set
F
F
F
F
F
...

Data 3
Test Set
I
I
I
F
I
...

Data 4
Test Set
M
M
F
I
M
...

Data 5
Test Set
F
F
F
I
I
...



For the last, we will calculate accuracy metric by comparing predicted_class and expected_class.

In [11]:
scores = list()
for i in range(len(test_collection)):
    correct = 0
    for j in range(len(test_collection[i])):
        if test_collection[i][j] == test_class_collection[i][j]:
            correct += 1
    
    accuracy = correct / len(test_collection[i])
    scores.append(accuracy)

print(f"Scores : {scores}\n")
print(f"Mean Accuracy : {sum(scores) / len(scores)}")

Scores : [0.511377245508982, 0.5089820359281437, 0.5221556886227545, 0.5317365269461077, 0.5305389221556887]

Mean Accuracy : 0.5209580838323353


From the result, it can be seen that this model has poor accuracy in detecting sex of Abalone. I hope that in the future I can improve this model even better.